In [ ]:
import numpy as np
from data_handler import StaticDataHandler
from model_builder import ModelBuilder
from constants import *
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
# from sklearn.model_selection import train_test_split
data = StaticDataHandler._load_h5py_data(H5PY_PATH)
print(len(data['inputs']))
print(len(data['targets']))
print("Train Test.....")
_train_data, _test_data = StaticDataHandler._train_split_test(split_ratio=0.2, input_data=data)
# X_train, X_test, y_train, y_test = train_test_split(data['inputs'], data['targets'], test_size=0.33, random_state=42)
print("Train Validation.....")
# X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.33, random_state=42)
_train_data, _validation_data = StaticDataHandler._train_split_test(split_ratio=0.2, input_data=_train_data)
mappings = StaticDataHandler._load_jsonfile_into_song(MAPPING_PATH)

In [ ]:
print("Data type:", type(mappings))
print("Total length:", len(mappings))
# print("Inputs Train Length", len(X_train))
# print("Inputs Validation Length", len(X_validation))
# print("Inputs Test Length", len(X_test))
# print("Total Length: ", len(X_train) + len(X_validation) + len(X_test))
print("Inputs Train Length", len(_train_data['inputs']))
print("Inputs Validation Length", len(_validation_data['inputs']))
print("Inputs Test Length", len(_test_data['inputs']))
print("Total Length: ", len(_train_data['inputs']) + len(_validation_data['inputs']) + len(_test_data['inputs']))

In [ ]:
# m = ModelBuilder()
# model = m._build_model()
model = load_model('model_checkpoint_epoch_20.keras')
model.summary()

In [4]:
# # Train
# history = model.fit(_train_data['inputs'], _train_data['targets'], 
#                     validation_data=(_validation_data['inputs'], _validation_data['targets']), 
#                     batch_size=BATCH_SIZE, epochs=NUMBER_EPOCH, validation_freq=5)
# # Evaluate
# test_error, test_accuracy = model.evaluate(_test_data['inputs'], _test_data['targets'], verbose=1)
# print("Accuracy on Test Set: ", test_accuracy)
# print("Error on Test Set: ", test_error)


# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((_train_data['inputs'], _train_data['targets']))
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

validation_dataset = tf.data.Dataset.from_tensor_slices((_validation_data['inputs'], _validation_data['targets']))
validation_dataset = validation_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((_test_data['inputs'], _test_data['targets']))
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    'model_checkpoint_epoch_{epoch}.keras',  # Save after each epoch
    save_best_only=True,                    # Save the model at every epoch, if False
    save_weights_only=False,                 # Save the entire model (architecture + weights + optimizer state)
    verbose=1
)
# Define learning rate scheduler
lr_scheduler = ReduceLROnPlateau(
    monitor='loss',   # Watch the loss for stopping condition
    factor=0.1,       # Reduce learning rate by 10% when the loss plateaus
    patience=3        # Wait for 3 epochs with no improvement
)
# Train
import threading
history = threading.Thread(target=lambda: model.fit(
                    train_dataset, 
                    validation_data=validation_dataset, 
                    epochs=NUMBER_EPOCH, initial_epoch=16, validation_freq=5, callbacks=[checkpoint_callback, lr_scheduler])).start()

In [ ]:
# Evaluate
# test_error, test_accuracy = model.evaluate(test_dataset, verbose=1)
# print("Accuracy on Test Set: ", test_accuracy)
# print("Error on Test Set: ", test_error)